In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display
import toml
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

def get_sheetdata(sheet_name):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load("/home/ciroshia/Downloads/github/comissao/.streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()

    df = pd.DataFrame(data)

    return df


def update_sheet(sheet_name,df):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load("/home/ciroshia/Downloads/github/comissao/.streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    worksheet.clear()
    set_with_dataframe(worksheet, df)

    return df


agendamento_path = "/home/ciroshia/Downloads/github/comissao/assets/compiled_agendamentos.xlsx"

agendamentos_df = pd.read_excel(agendamento_path)
procedimentos_df = get_sheetdata("procedimentos_padronizados")
prestadora_df = get_sheetdata("base_prestadoras")
comissao_df = get_sheetdata("comissoes")
tipo_prestadora_df = get_sheetdata("tipo_prestadora")

In [1]:
import os
import glob
import pandas as pd

def compile_agendamentos(folder_path):

    pattern = os.path.join(folder_path, 'agendamentos*.xlsx')
    file_list = glob.glob(pattern)
    
    if not file_list:
        print(f"No files found matching the pattern in folder: {folder_path}")
        return

    # List to store DataFrames from each file
    df_list = []
    
    for file in file_list:
        try:
            df = pd.read_excel(file)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    if not df_list:
        print("No valid DataFrames were loaded.")
        return

    compiled_df = pd.concat(df_list, ignore_index=True)
    

    compiled_df.drop_duplicates(inplace=True)
    

    output_file = os.path.join(folder_path, 'compiled_agendamentos.xlsx')
    
    try:

        compiled_df.to_excel(output_file, index=False)
        print(f"Compiled file saved successfully at: {output_file}")
    except Exception as e:
        print(f"Error saving compiled file: {e}")

compile_agendamentos('/home/ciroshia/Downloads/github/comissao/assets')


Compiled file saved successfully at: /home/ciroshia/Downloads/github/comissao/assets/compiled_agendamentos.xlsx


In [4]:
base_limpa = agendamentos_df.loc[agendamentos_df["Unidade do agendamento"] != 'plástica']
base_limpa = base_limpa.loc[base_limpa["Unidade do agendamento"] != 'homa']
base_limpa = pd.merge(base_limpa, prestadora_df, left_on="Prestador", right_on="nome_prestadora", how="left")
base_limpa = pd.merge(base_limpa, procedimentos_df, left_on="Procedimento", right_on="procedimento_crm", how="left")

# base_limpa = pd.merge(base_limpa, comissao_df, left_on=["procedimento_padronizado","tipo_prestadora"], right_on=["procedimento_padronizado","tipo_prestadora"], how="left")
# base_limpa = pd.merge(base_limpa, tipo_prestadora_df, left_on=["tipo_prestadora"], right_on=["tipo_prestadora"], how="left")

colunas = ['Procedimento', 'nome_prestadora','tipo_prestadora','procedimento_padronizado']

base_limpa = base_limpa.loc[~base_limpa["nome_prestadora"].isna(),colunas]
base_para_subir = base_limpa[["procedimento_padronizado","tipo_prestadora"]].drop_duplicates()
base_para_subir["comissão"] = 0
base_para_subir = base_para_subir.loc[~base_para_subir["procedimento_padronizado"].isna()]
base_para_subir.head(50)

,procedimento_padronizado,tipo_prestadora,comissão
0,DRENAGEM (30 MIN),PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
2,RADIOFREQUÊNCIA,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
3,DRENAGEM,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
20,SCULPTRA,PRESTAÇÃO DE SERVIÇOS - BIOMEDICA,0
22,LIMPEZA DE PELE,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
53,RESERVA DE HORÁRIO,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
57,LAVIEEN,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
60,PRÓ-SHAPE,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
85,CARBOXITERAPIA,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
96,PEELING DE DIAMANTE,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0


In [5]:
update_sheet("comissoes",base_para_subir)

,procedimento_padronizado,tipo_prestadora,comissão
0,DRENAGEM (30 MIN),PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
2,RADIOFREQUÊNCIA,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
3,DRENAGEM,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
20,SCULPTRA,PRESTAÇÃO DE SERVIÇOS - BIOMEDICA,0
22,LIMPEZA DE PELE,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
...,...,...,...
55496,UP BUMBUM,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
58111,PRÓ-LIPO,PRESTAÇÃO DE SERVIÇOS - MÉDICO DERMATO,0
62432,RADIESSE,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
73191,CO2 COLO,PRESTAÇÃO DE SERVIÇOS - BIOMEDICA,0
